# [문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough
- itemgetter()


In [ ]:
## 1. 모듈 import
from operator import itemgetter
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import trim_messages
from langchain_core.chat_history import (
    BaseChatMessageHistory, 
    InMemoryChatMessageHistory
)


## 2. .env 파일에서 환경변수 읽어오기
load_dotenv()

## 3. 세션별 대화 히스토리를 저장할 임시 메모리 저장소
store = {}

## 4. 함수 정의: 세션 ID에 따라 대화 히스토리 반환
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 5. prompt template 정의
messages = [
    ('system', '''당신은 이력서 작성 컨설턴트입니다.
아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
문장은 자연스럽고 매끄럽게 작성합니다.'''),
	('placeholder', '{chat_history}'),
	('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(messages=messages)

## 6. ChatOpenAI 인스턴스 생성: 모델 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

## 7. trim 설정.
trimmer = trim_messages(
    max_tokens=65,
    strategy='last',
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on='human',
)

## 8. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history')| trimmer)
    |prompt 
    | llm 
)

## 9. chain에 대화 히스토리 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key='query',
    history_messages_key='chat_history'
)

## 10. chain 실행
while True:
    query = input('이력서 작성 컨설턴트입니다. 질문하세요. [종료: S] >>> ')

    if query.upper() == 'S':
        break

    with_message_history.invoke(
        {'query': query},
        config={'configurable': {'session_id': '1234'}}      
    )
    print('\n'+'='*50 + '\n')

물론입니다. 이력서를 작성하기 위해 귀하의 학력, 경력, 기술 및 기타 관련 정보를 필요로 합니다. 제공해주실 수 있는 모든 세부사항을 공유해주시면 감사하겠습니다.

이력서를 작성하기 위해 다음과 같은 정보를 제공해 주시면 좋습니다:

1. 개인 정보: 이름, 연락처(전화번호, 이메일), 주소(선택 사항).
2. 경력 사항: 현재 및 이전 직장의 이름, 직급, 근무 기간, 주요 업무 및 성과.
3. 학력: 최종 학력 및 기타 학력 정보(졸업 학교명, 전공, 졸업 연도).
4. 기술: 숙련된 소프트웨어, 언어, 기타 전문 기술.
5. 자격증 및 수상 경력: 관련 자격증이나 수상 경력.
6. 기타 활동: 인턴십, 자원봉사, 동아리 활동 등.
7. 목표나 자기소개서(선택 사항): 지원하는 직무에 관련된 자신의 목표와 강점을 간략히 소개.

이 정보를 바탕으로 이력서를 작성해 드릴 수 있습니다!

---
**이력서**

**개인 정보**  
이름: 황정민  

**경력 요약**  
- 배우로서 약 20년의 경력을 가지고 있으며, 다양한 장르와 배역에서 깊이 있는 연기를 선보였음.
- 다수의 작품에서 주연을 맡아 관객과 평단을 모두 사로잡은 연기력을 인정받음.

**교육**  
- 연기 전공, 서울예술대학교 (졸업연도 기재 필요)

**경험**  
1. **영화 및 드라마 출연**  
   - 다수의 유명 영화 및 드라마에서 주연을 맡으며 신뢰 있는 연기자로 자리매김.
   - 다양한 캐릭터에 몰입하여 역할 소화, 극의 완성도를 높이는 데 기여.

2. **수상 경력**  
   - 다수의 남우주연상 수상: 연기력을 공인받고 인정을 받은 각종 영화제에서 최고 배우로 선정.
   - 특정 년도 및 영화제 수상 내역 필요.

**기술 및 전문성**  
- **메소드 연기**: 캐릭터의 심리와 환경을 철저히 연구하여 몰입도가 높고 진정성 있는 연기를 펼침.  
- 다양한 장르 소화 능력: 드라마, 액션, 코미디 등 여러 장르의 배역을 소화하며, 장르를 초월한 매력을 발휘.


In [ ]:
get_history('1234')

InMemoryChatMessageHistory(messages=[HumanMessage(content='자기소개서를 어떻게 작성하면되?', additional_kwargs={}, response_metadata={}), AIMessage(content='자기소개서를 작성할 때에는 자신의 강점과 경험을 효과적으로 드러내면서, 지원하는 회사와 직무에 적합한 인재임을 어필하는 것이 중요합니다. 아래에 자기소개서 작성 방식과 팁을 안내해 드리니 참고하셔서 자신만의 이야기를 자연스럽게 풀어보세요.\n\n1. 시작은 본인 소개와 목표 제시\n- 간단하게 본인에 대한 소개와 지원 동기, 그리고 앞으로의 목표를 명확히 전달하세요.\n- 예를 들어, “저는 △△ 분야에 열정을 품고 성장해온 ○○입니다. 이번 기회를 통해 제 역량을 발휘하며 회사와 함께 성장', additional_kwargs={}, response_metadata={'finish_reason': 'length', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f'}, id='run-82bab0a4-d555-4a8e-a72d-9b10f6fd27c0-0')])

In [ ]:
for info in get_history('1234').messages:
    print(info.content)
    print('*' * 50)

자기소개서를 어떻게 작성하면되?
**************************************************
자기소개서를 작성할 때에는 자신의 강점과 경험을 효과적으로 드러내면서, 지원하는 회사와 직무에 적합한 인재임을 어필하는 것이 중요합니다. 아래에 자기소개서 작성 방식과 팁을 안내해 드리니 참고하셔서 자신만의 이야기를 자연스럽게 풀어보세요.

1. 시작은 본인 소개와 목표 제시
- 간단하게 본인에 대한 소개와 지원 동기, 그리고 앞으로의 목표를 명확히 전달하세요.
- 예를 들어, “저는 △△ 분야에 열정을 품고 성장해온 ○○입니다. 이번 기회를 통해 제 역량을 발휘하며 회사와 함께 성장
**************************************************
